In [7]:
dict={"uno":1,"dos":2,"res":3}




In [10]:
print dict

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(dict)? (<ipython-input-10-ad2a0fe8b542>, line 1)

In [ ]:
print(dict)

In [ ]:
dict["uno"]=100


In [ ]:
print (dict)

In [ ]:
for i in dict:
    print(i)

In [11]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

df=pd.read_csv("/Users/mushirih/Desktop/stuff/jupyter/Loan-Prediction-Dataset-master/train.csv")

In [12]:
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [13]:
df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [14]:
df['Property_Area'].value_counts()

Semiurban    233
Urban        202
Rural        179
Name: Property_Area, dtype: int64

In [ ]:
df['Credit_History'].value_counts()


In [ ]:
df['ApplicantIncome'].hist(bins=50)

In [ ]:
df.boxplot(column="ApplicantIncome")

In [ ]:
df.boxplot(column="ApplicantIncome", by="Education")

In [ ]:
df["LoanAmount"].hist(bins=50)

In [ ]:
df.boxplot(column="LoanAmount")

In [ ]:
temp1 = df["Credit_History"].value_counts(ascending=True)
temp2=df.pivot_table(values="Loan_Status",index=['Credit_History'],aggfunc=lambda x:x.map({'Y':1,'N':0}).mean())
print('Frequency Table for credit history')
print(temp1)

print("\n Probability of getting a loan in each credit history class")
print (temp2)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8,4))
ax1=fig.add_subplot(121)
ax1.set_xlabel('Credit History')
ax1.set_ylabel('Count of applicants')
ax1.set_title("Applicants by credit history")
temp1.plot(kind='bar')

ax2=fig.add_subplot(122)
temp2.plot(kind='bar')
ax2.set_xlabel('Credit History')
ax2.set_ylabel("Probability of getting a loan")
ax2.set_title("Probability of getting loan by credit history")

In [ ]:
temp3 = pd.crosstab(df['Credit_History'],df['Loan_Status'])
temp3.plot(kind='bar', stacked=True, color=['red','blue'])

In [ ]:
df.apply(lambda x: sum(x.isnull()),axis=0)

In [ ]:
df['']

In [ ]:
df['LoanAmount'].fillna(df['LoanAmount'].mean(),inplace=True)

In [ ]:
df['Self_Employed'].fillna(df['Self_Employed'].mode(),inplace=True)

In [ ]:
table = df.pivot_table(values="LoanAmount", index="Self_Employed",columns="Education",aggfunc=np.median)

def fage(x):
    return table.loc[x['Self_Employed'],x['Education']]

df['LoanAmount'].fillna(df[df['LoanAmount'].isnull()].apply(fage(df),axis=1),inplace=True)

In [ ]:
df['LoanAmount_log']=np.log(df['LoanAmount'])
df['LoanAmount_log'].hist(bins=20)

In [ ]:
df['TotalIncome']= df['ApplicantIncome']+df['CoapplicantIncome']
df['TotalIncome_log']=np.log(df['TotalIncome'])
df['LoanAmount_log'].hist(bins=20)

In [ ]:
df['Gender'].fillna(df['Gender'].mode()[0],inplace=True)
df['Married'].fillna(df['Married'].mode()[0],inplace=True)
df['Dependents'].fillna(df['Dependents'].mode()[0],inplace=True)
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0],inplace=True)
df['Credit_History'].fillna(df['Credit_History'].mode()[0],inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

var_mod=['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status']
le= LabelEncoder()
for i in var_mod:
    df[i]=le.fit_transform(df[i])
    
df.dtypes

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics


def classification_model(model, data, predictors, outcome):
    model.fit(data[predictors],data[outcome])
    
    predictions = model.predict(data[predictors])
    
    accuracy=metrics.accuracy_score(predictions,data[outcome])
    print("Accuracy: %s" %"{0:.3%}".format(accuracy))
          
    kf = KFold()
    error=[]
    
    for train, test in kf.split(data):
        train_predictors =(data[predictors].iloc[train,:])
        
        train_target = data[outcome].iloc[train]
        
        model.fit(train_predictors, train_target)
        
        error.append(model.score(data[predictors].iloc[test,:],data[outcome].iloc[test]))
        
    print ("Cross validation score: %s" % "{0:.3%}".format(np.mean(error)))

    model.fit(data[predictors, data[outcome]])
          

In [ ]:
# Logistic regression

outcome_var = 'Loan_Status'
model = LogisticRegression()
predictor_var=['Credit_History']
classification_model(model, df,predictor_var, outcome_var)









In [ ]:
# Decision tree

model = DecisionTreeClassifier()

predictor_var = ['Credit_History','Gender','Married','Education']

classification_model(model, df, predictor_var, outcome_var)

In [ ]:
predictor_var = ['Credit_History','Loan_Amount_Term','LoanAmount_log']
classification_model(model, df, predictor_var, outcome_var)

In [ ]:
# Random Forest

model = RandomForestClassifier(n_estimators=100)
predictor_var=["Gender","Married","Dependents","Education","Self_Employed","Loan_Amount_Term","Credit_History","Property_Area","LoanAmount_log","TotalIncome_log"]
classification_model(model, df, predictor_var, outcome_var)

In [ ]:
feature_importance = pd. Series(model.feature_importances_, index= predictor_var).sort_values(ascending=False)
print(feature_importance)

In [ ]:
# Random Forest top 5 features

model = RandomForestClassifier(n_estimators=25, min_samples_split=25, max_depth=7, max_features=1)
predictor_var=["Dependents","Credit_History","Property_Area","LoanAmount_log","TotalIncome_log"]
classification_model(model, df, predictor_var, outcome_var)

In [ ]:
print (df)


In [ ]:
print(pd.crosstab(df['Gender'],df['Married']))

In [ ]:
pd.pivot_table(df[['Gender','Married']],index=df['Gender'],columns=df['Married'],aggfunc=lambda x: len(x.unique()))


In [ ]:
pd.re